In [ ]:
!pip install pyopencl

### Kernel

In [ ]:
%%writefile program.cl

__kernel void matrix_mul(__global int* n, __global int* a, __global int* b, __global int* c) {
	int id_x = get_global_id(0);
	int id_y = get_global_id(1);
	int width = n[0];

	c[id_y*width + id_x] = 0;

	for(int i = 0; i < width; i++)
		c[id_y*width + id_x] += a[id_y*width + i] * b[i*width + id_x];

}

### Runtime

In [ ]:
import numpy as np
import pyopencl as cl

np.random.seed(0)

WIDTH = 250
RANGE = 5

n = np.array([WIDTH], dtype=np.int32)
a = np.random.randint(-RANGE, RANGE, size=(WIDTH, WIDTH), dtype=np.int32)
b = np.random.randint(-RANGE, RANGE, size=(WIDTH, WIDTH), dtype=np.int32)

c = np.zeros_like(a)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
n_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=n)
a_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)
b_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=b)
c_buf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=c)

program_src = open("program.cl", "r").read()
program = cl.Program(ctx, program_src)
program.build()
matrix_mul = program.matrix_mul

matrix_mul(queue, c.shape, None, n_buf, a_buf, b_buf, c_buf)
cl.enqueue_copy(queue, c, c_buf)

print("Results matching:", np.allclose(a @ b, c))